# Phase 2: Exploratory Data Analysis (Dataset 2)

---
## Table of Contents
1. [Introduction](#1-introduction)
2. [Data Loading](#2-loading-libraries-and-dataset)
3. [Data Exploration](#3-data-exploration)

---

## 1. Introduction
In this notebook, we perform an exploratory data analysis (EDA) on **Dataset 2**.  
Goals of this notebook:
- Understand the structure of the dataset.  
- Check for missing values and inconsistencies.  
- Perform exploratory statistics and visualizations.  
- Extract insights that will guide preprocessing and modeling.  


## 2. Loading Libraries and Dataset

In [6]:
# Import necessary libraries
import pandas as pd
import ast


In [7]:
# Loading the datasets
credits_2 = pd.read_csv('../data/raw/dataset2/tmdb_5000_credits.csv')
movies_df_2 = pd.read_csv('../data/raw/dataset2/tmdb_5000_movies.csv')

## 3. Data Exploration

In [8]:
# Merge using common ID
df = movies_df_2.merge(credits_2, left_on='id', right_on='movie_id', how='left')

# Preview
print(f"Movies shape: {movies_df_2.shape}")
print(f"Credits shape: {credits_2.shape}")
print(f"Merged shape: {df.shape}")
df.head(1)

Movies shape: (4803, 20)
Credits shape: (4803, 4)
Merged shape: (4803, 24)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


- Merged dataset has **4803 movies × 24 columns** → duplicates in `title_x` vs. `title_y`, `id` vs. `movie_id`.  
- **Cleaning steps to apply**:  
  1. **Drop duplicates**: keep one of `title_x`/`title_y`, and one of `id`/`movie_id`.  
  2. **Remove irrelevant columns** for recommendation: `homepage`, `status`, `tagline` (sparse or uninformative).  
  3. **Parse JSON-like fields** (`genres`, `keywords`, `cast`, `crew`, `production_companies`, `spoken_languages`) into clean lists of strings.  
  4. **Check `budget` and `revenue`**: many zero values → decide whether to keep, log-transform, or drop.  
  5. **Normalize text columns** (`overview`, `title`) → lowercase, remove punctuation/stopwords.  
  6. **Handle missing values**: drop rows with excessive nulls, fill with placeholders where appropriate.  
  7. Ensure **numeric columns** (`vote_average`, `vote_count`, `popularity`) are cast to proper types.  


In [9]:
df.drop(columns=['budget', 'homepage', 'original_language','original_title', 'production_companies', 
                     'production_countries', 'revenue', 'runtime','spoken_languages', 'status', 'title_y','movie_id'], inplace=True)

In [10]:
# functions to parse jsons
def parse_names(obj_str, key='name'):
    try:
        data = ast.literal_eval(obj_str)
        return [d[key] for d in data if key in d]
    except Exception:
        return []

def get_director(crew_str):
    try:
        crew_list = ast.literal_eval(crew_str)
        for member in crew_list:
            if member.get('job') == 'Director':
                return member.get('name')
        return None
    except Exception:
        return None


In [11]:
# apply the functions then drop the old columns
df['genre_names'] = df['genres'].apply(parse_names)
df['keyword_names'] = df['keywords'].apply(parse_names)
df['cast_names'] = df['cast'].apply(parse_names)
df['director'] = df['crew'].apply(get_director)

df['release_year'] = df['release_date'].astype(str).str[:4]

df.drop(columns=['genres','cast','crew','keywords','release_date'],inplace=True)

In [12]:
df.head(1)

,id,overview,popularity,tagline,title_x,vote_average,vote_count,genre_names,keyword_names,cast_names,director,release_year
0,19995,"In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.,Avatar,7.2,11800,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,2009


In [14]:
df['overview'] = df['overview'].fillna('')
df_cleaned = df.rename(columns={'title_x': 'title', 'keyword_names':'keywords', 'genre_names':'genres'})
df_cleaned.to_csv('../data/processed/dataset2/cleaned_movies2.csv', index=False)


In [15]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   overview      4803 non-null   object 
 2   popularity    4803 non-null   float64
 3   tagline       3959 non-null   object 
 4   title         4803 non-null   object 
 5   vote_average  4803 non-null   float64
 6   vote_count    4803 non-null   int64  
 7   genres        4803 non-null   object 
 8   keywords      4803 non-null   object 
 9   cast_names    4803 non-null   object 
 10  director      4773 non-null   object 
 11  release_year  4803 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 450.4+ KB


- Dataset: 4803 movies × 12 columns.  
- Missing: over 17% of the taglines are missing while less than 1% of movies have a missing director.
- JSON-like cols parsed → string lists (genres, keywords, cast).  


All missing values will be dealt with in the preprocessing stage. No visualization was done here since the main focus was to clean the data.